In [43]:
import pickle
import time
import sys
from math import radians, cos, sin, asin, sqrt

import pandas as pd
import numpy as np

In [3]:
edges =pd.read_csv("../backend/edge_list_pairs.csv")
edges.head()

/home/teadove/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3427: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id1,id2,dist,highway,surface,lit,lanes,width
0,2521296427,271012100,39.741627,tertiary,asphalt,no,2,2
1,271012100,96919511,126.400589,tertiary,asphalt,yes,1,2
2,31447629,2477000557,72.587743,tertiary,asphalt,yes,1,2
3,31447354,31448413,94.910701,tertiary,asphalt,yes,1,2
4,31448413,31448414,23.660839,tertiary,asphalt,yes,1,2


In [9]:
with open('../samples/archived/nodes.p', 'rb') as f:
    nodes = pickle.load(f)

print(nodes[0])
print(len(nodes))
print(sys.getsizeof(nodes))
print(type(nodes))

{'id': '26999673', 'lat': '55.7788343', 'lon': '37.6537207'}
3889497
34191104
<class 'list'>


In [42]:
"""
Минимум: 75
Максимум: 317
Генерация np матрицы высот
"""
cell_size = 0.00083333333333333
lat_start = 55.0
lon_start = 35.0

matrix_of_heights = []
with open('../data/srtm/srtm_44_01.asc') as f:
    data_str = f.readlines()

for row in data_str[6:]:
    row_list = list(map(int, row.split()))
    matrix_of_heights.append(row_list)

del row_list
del data_str

matrix_of_heights = np.array(matrix_of_heights, np.ushort)

In [53]:
def haversine(lat1: float, lon1: float, lat2: float, lon2: float):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371000 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [68]:
def get_height(lat: float, lon: float) -> float:
    """
    Получение высоты над уровнем моря по координатам
    :param lat: Долгота точки
    :param lon: Широта точки
    :return: Высота над уровнем моря
    """
    semi_x = (lat - lat_start) / cell_size
    semi_y = (lon - lon_start) / cell_size
    if semi_x > 6000 or semi_y > 6000:
        raise Exception('Out of bounds, not in Moscow')

    points = [matrix_of_heights[int(np.floor(semi_x)), int(np.floor(semi_y))],
              matrix_of_heights[int(np.floor(semi_x)), int(np.ceil(semi_y))],
              matrix_of_heights[int(np.ceil(semi_x)), int(np.floor(semi_y))],
              matrix_of_heights[int(np.ceil(semi_x)), int(np.ceil(semi_y))]]
    floor_lat = np.floor(semi_x) * cell_size + lat_start
    floor_lon = np.floor(semi_y) * cell_size + lon_start
    ceil_lat = np.ceil(semi_x) * cell_size + lat_start
    ceil_lon = np.ceil(semi_y) * cell_size + lon_start
    coordinates = [[floor_lat, floor_lon], [floor_lat, ceil_lon], [ceil_lat, floor_lon],
                   [ceil_lat, ceil_lon]]
    idx_min, min_ = 0, 200
    for idx, point in enumerate(coordinates):
        dist_ = haversine(lat, lon, *point)
        if dist_ < min_:
            min_ = dist_
            idx_min = idx
    triangle = [points[idx_min-1], points[idx_min], points[(idx_min+1)%4]]
    return sum(triangle)/3


get_square(55.7558, 37.6173)

157.66666666666666